In [1]:
# import system modules
import sys
import os
os.system('for a in /sys/bus/pci/devices/*; do echo 0 | tee -a $a/numa_node>/dev/null; done') # get rid of NUMA node warnings in my docker: https://github.com/tensorflow/tensorflow/issues/42738
import gc

# import standard numerical modules
import numpy as np
import math

# import machine learning modules
import tensorflow as tf
import keras.backend as K

gpu = tf.config.list_physical_devices('GPU') # make sure GPU usage is enabled
tf.config.experimental.set_virtual_device_configuration(gpu[0], [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=int(7.5*1024))]) # make sure tensorflow doesn't use too much memorty and crashes
print(gpu) 

import DCTR


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


# load data

In [2]:
# directory with pre converted lhe files as numpy arrays
data_dir = '../Data' # modify as needed


In [3]:
# Load POWHEG hvq x0 datasets
# x0_nrm for training, x0_plt and x0_plt_nrm for calculating stats used to decide which model performs best
# only contain tt-pair; every event has order: 
    # tt-pair, top, anti-top
# every particle has arguments: 
    # [pt, y, phi, mass, eta, E, PID, w, theta]
    # [0 , 1, 2  , 3   , 4  , 5, 6  , 7, 8    ]

# POWHEG hvq
x0_nrm = []
x0_nrm = DCTR.load_dataset(f'{data_dir}/POWHEG_hvq/13TeV/01-02_normed_converted_lhe.npz', i=3)[:9553938] # 7754952 NLO evts have the same sum of wgts as 9553938 NNLO evts | figured out 3 cells below in commented out code
print(f'POWHEG hvq x0_nrm.shape:     {x0_nrm.shape}')

# plotting data; different from training data; for calculating stats
x0_plt = []
x0_plt = DCTR.load_dataset(f'{data_dir}/POWHEG_hvq/13TeV/03-04_converted_lhe.npz', i=3)[:int(2e7)]
print(f'POWHEG hvq x0_plt.shape:     {x0_plt.shape}')

x0_plt_nrm = []
x0_plt_nrm = DCTR.load_dataset(f'{data_dir}/POWHEG_hvq/13TeV/03-04_normed_converted_lhe.npz', i=3)[:int(2e7)]
print(f'POWHEG hvq x0_plt_nrm.shape: {x0_plt_nrm.shape}')



POWHEG hvq x0_nrm.shape:     (9553938, 3, 9)
POWHEG hvq x0_plt.shape:     (20000000, 3, 9)
POWHEG hvq x0_plt_nrm.shape: (20000000, 3, 9)


In [4]:
# MiNNLO x1
# training data
x1_nrm = []
x1_nrm = DCTR.load_dataset(f'{data_dir}/MiNNLO/converted_with_13TeV_NLO/normed_converted_lhe.npz', i=3)
print(f'MiNNLO all particles x1_nrm.shape: {x1_nrm.shape}')

# plotting data
x1_plt = []
x1_plt = DCTR.load_dataset(f'{data_dir}/MiNNLO/converted_with_13TeV_NLO/converted_lhe.npz', i=3)
print(f'MiNNLO all particles x1_plt.shape: {x1_plt.shape}')


MiNNLO all particles x1_nrm.shape: (9553938, 3, 9)
MiNNLO all particles x1_plt.shape: (9553938, 3, 9)


In [5]:
# get normalized event generator weights
x0_wgt = x0_nrm[:, 0, 7].copy()

x0_plt_wgt = x0_plt_nrm[:, 0, 7].copy() 

x1_wgt = x1_nrm[:, 0, 7].copy()


In [6]:
# used to check how many NLO evts have the same sum of wgts as the 9553938 NNLO evts
# print(f'x0 wgt sum: {np.sum(x0_wgt[:7754952])}')
# print(f'x1 wgt sum: {np.sum(x1_wgt)}')

# check that sum of wgt is same for both classes
print(f'x0 wgt sum: {np.sum(x0_wgt)}')
print(f'x1 wgt sum: {np.sum(x1_wgt)}')


x0 wgt sum: 9476758.0
x1 wgt sum: 7692656.0


In [7]:
# delete eta (pseudorapidity) and Energy -> Train only with [pt, y, phi, m, PID]

# delete energy
x0_nrm = np.delete(x0_nrm, 5, -1)
x0_plt_nrm = np.delete(x0_plt_nrm, 5, -1)
x1_nrm = np.delete(x1_nrm, 5, -1)

# delete eta
x0_nrm = np.delete(x0_nrm, 4, -1)
x0_plt_nrm = np.delete(x0_plt_nrm, 4, -1)
x1_nrm = np.delete(x1_nrm, 4, -1)


In [8]:
# prep arrays for training
x_train, x_val, y_train, y_val, wgt_train, wgt_val = DCTR.prep_arrays(x0_nrm, x1_nrm, val=0.25)

# bring into shape for training loop
train_data = (x_train, y_train, x_val, y_val, wgt_train, wgt_val)
plt_data = (x0_plt , x0_plt_nrm, x1_plt, x1_wgt)


2024-04-11 15:52:44.588580: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-11 15:52:45.228423: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 7475 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2080 SUPER, pci bus id: 0000:09:00.0, compute capability: 7.5
2024-04-11 15:52:45.257950: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 1719708840 exceeds 10% of free system memory.


# training loop

In [16]:
K.clear_session() 
print(gc.collect()) # cpu gabage collection


100


In [10]:
train_dir = './train_20240411' # where to save models during training


In [11]:
# start training loop
''' train_loop() necessary arguments
train_data, plt_data

default arguments:
model=None, lowest_chi2 = 1e6, train_dir = '/tf/home/gdrive/_STUDIUM_/DCTR_Paper/train',
batch_sizes=[4*8192, 8*8192, 16*8192, 32*8192], repeat=5, super_epochs=35, super_patience = 5, epochs = 8, starting_super_epoch = 1, 
input_dim=5, Phi_sizes = (100,100,128), F_sizes = (128,100,100), loss = 'mse', dropout=0.0, l2_reg=0.0, 
Phi_acts=('linear', 'gelu', 'gelu'), F_acts=('gelu', 'gelu', 'linear'), output_act='sigmoid', learning_rate=0.001

returns: best_model_list, lowest_chi2_list, lowest_loss_list
'''
best_model_list, lowest_chi2_list, lowest_loss_list = DCTR.train_loop(train_data, plt_data, batch_sizes=[24*8192, 32*8192], repeat=7, super_epochs=5, 
                                                                      train_dir = train_dir, epochs=10, learning_rate=0.001)


best_model = best_model_list[-1]
lowest_chi2 = lowest_chi2_list[-1]


starting super_epoch 1

starting training with batch_size: 196608 and 10 epochs
starting with weights from model: None
starting run 0 of super_epoch 1 with batch_size 196608


2024-04-11 15:54:22.729410: I tensorflow/stream_executor/cuda/cuda_driver.cc:739] failed to allocate 7.30G (7838105600 bytes) from device: CUDA_ERROR_OUT_OF_MEMORY: out of memory
2024-04-11 15:54:22.859469: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 7475 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2080 SUPER, pci bus id: 0000:09:00.0, compute capability: 7.5


reset neural network weights


2024-04-11 15:54:33.476945: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: ./train_20240411/super_epoch_1/run_0/s-1_b-196608_r-0.tf/assets
INFO:tensorflow:Assets written to: ./train_20240411/super_epoch_1/run_0/s-1_b-196608_r-0.tf/assets
INFO:tensorflow:Assets written to: ./train_20240411/super_epoch_1/run_0/s-1_b-196608_r-0.tf/assets
INFO:tensorflow:Assets written to: ./train_20240411/super_epoch_1/run_0/s-1_b-196608_r-0.tf/assets
INFO:tensorflow:Assets written to: ./train_20240411/super_epoch_1/run_0/s-1_b-196608_r-0.tf/assets
INFO:tensorflow:Assets written to: ./train_20240411/super_epoch_1/run_0/s-1_b-196608_r-0.tf/assets
INFO:tensorflow:Assets written to: ./train_20240411/super_epoch_1/run_0/s-1_b-196608_r-0.tf/assets
INFO:tensorflow:Assets written to: ./train_20240411/super_epoch_1/run_0/s-1_b-196608_r-0.tf/assets
INFO:tensorflow:Assets written to: ./train_20240411/super_epoch_1/run_0/s-1_b-196608_r-0.tf/assets
INFO:tensorflow:Assets written to: ./train_20240411/super_epoch_1/run_0/s-1_b-196608_r-0.tf/assets
INFO:tenso

...

INFO:tensorflow:Assets written to: ./train_20240411/super_epoch_1/run_4/s-1_b-262144_r-4.tf/assets
INFO:tensorflow:Assets written to: ./train_20240411/super_epoch_1/run_4/s-1_b-262144_r-4.tf/assets
INFO:tensorflow:Assets written to: ./train_20240411/super_epoch_1/run_4/s-1_b-262144_r-4.tf/assets
INFO:tensorflow:Assets written to: ./train_20240411/super_epoch_1/run_4/s-1_b-262144_r-4.tf/assets
INFO:tensorflow:Assets written to: ./train_20240411/super_epoch_1/run_4/s-1_b-262144_r-4.tf/assets
INFO:tensorflow:Assets written to: ./train_20240411/super_epoch_1/run_4/s-1_b-262144_r-4.tf/assets
INFO:tensorflow:Assets written to: ./train_20240411/super_epoch_1/run_4/s-1_b-262144_r-4.tf/assets
INFO:tensorflow:Assets written to: ./train_20240411/super_epoch_1/run_4/s-1_b-262144_r-4.tf/assets
INFO:tensorflow:Assets written to: ./train_20240411/super_epoch_1/run_4/s-1_b-262144_r-4.tf/assets
INFO:tensorflow:Assets written to: ./train_20240411/super_epoch_1/run_4/s-1_b-262144_r-4.tf/assets
INFO:tenso

2024-04-11 16:22:19.865328: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 7475 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2080 SUPER, pci bus id: 0000:09:00.0, compute capability: 7.5


INFO:tensorflow:Assets written to: ./train_20240411/super_epoch_1/run_5/s-1_b-262144_r-5.tf/assets
INFO:tensorflow:Assets written to: ./train_20240411/super_epoch_1/run_5/s-1_b-262144_r-5.tf/assets
INFO:tensorflow:Assets written to: ./train_20240411/super_epoch_1/run_5/s-1_b-262144_r-5.tf/assets
INFO:tensorflow:Assets written to: ./train_20240411/super_epoch_1/run_5/s-1_b-262144_r-5.tf/assets
INFO:tensorflow:Assets written to: ./train_20240411/super_epoch_1/run_5/s-1_b-262144_r-5.tf/assets
INFO:tensorflow:Assets written to: ./train_20240411/super_epoch_1/run_5/s-1_b-262144_r-5.tf/assets
INFO:tensorflow:Assets written to: ./train_20240411/super_epoch_1/run_5/s-1_b-262144_r-5.tf/assets
INFO:tensorflow:Assets written to: ./train_20240411/super_epoch_1/run_5/s-1_b-262144_r-5.tf/assets
INFO:tensorflow:Assets written to: ./train_20240411/super_epoch_1/run_5/s-1_b-262144_r-5.tf/assets
INFO:tensorflow:Assets written to: ./train_20240411/super_epoch_1/run_5/s-1_b-262144_r-5.tf/assets
INFO:tenso

2024-04-11 16:23:55.106665: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 7475 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2080 SUPER, pci bus id: 0000:09:00.0, compute capability: 7.5


INFO:tensorflow:Assets written to: ./train_20240411/super_epoch_1/run_6/s-1_b-262144_r-6.tf/assets
INFO:tensorflow:Assets written to: ./train_20240411/super_epoch_1/run_6/s-1_b-262144_r-6.tf/assets
INFO:tensorflow:Assets written to: ./train_20240411/super_epoch_1/run_6/s-1_b-262144_r-6.tf/assets
INFO:tensorflow:Assets written to: ./train_20240411/super_epoch_1/run_6/s-1_b-262144_r-6.tf/assets
INFO:tensorflow:Assets written to: ./train_20240411/super_epoch_1/run_6/s-1_b-262144_r-6.tf/assets
INFO:tensorflow:Assets written to: ./train_20240411/super_epoch_1/run_6/s-1_b-262144_r-6.tf/assets
INFO:tensorflow:Assets written to: ./train_20240411/super_epoch_1/run_6/s-1_b-262144_r-6.tf/assets
INFO:tensorflow:Assets written to: ./train_20240411/super_epoch_1/run_6/s-1_b-262144_r-6.tf/assets
INFO:tensorflow:Assets written to: ./train_20240411/super_epoch_1/run_6/s-1_b-262144_r-6.tf/assets
INFO:tensorflow:Assets written to: ./train_20240411/super_epoch_1/run_6/s-1_b-262144_r-6.tf/assets
INFO:tenso

In [17]:
K.clear_session() 
print(gc.collect()) # cpu gabage collection

# continue training loop with best model as starting point and smaller batch_ sizes and less repeats

model_list, chi2_list, loss_list = DCTR.train_loop(train_data, plt_data, model=best_model, lowest_chi2=lowest_chi2, batch_sizes=[16*8192, 24*8192], repeat=4, super_epochs=4, 
                                                   starting_super_epoch=6, train_dir = train_dir, epochs=10, learning_rate=0.001)


best_model_list.append(model_list)
lowest_chi2_list.append(chi2_list)

best_model = best_model_list[-1]
lowest_chi2 = lowest_chi2_list[-1]


100
starting super_epoch 6

starting training with batch_size: 131072 and 10 epochs
starting with weights from model: ./train_20240411/super_epoch_4/run_2/s-4_b-262144_r-2.tf
starting run 0 of super_epoch 6 with batch_size 131072
loaded neural network model: ./train_20240411/super_epoch_4/run_2/s-4_b-262144_r-2.tf
INFO:tensorflow:Assets written to: ./train_20240411/super_epoch_6/run_0/s-6_b-131072_r-0.tf/assets
INFO:tensorflow:Assets written to: ./train_20240411/super_epoch_6/run_0/s-6_b-131072_r-0.tf/assets
INFO:tensorflow:Assets written to: ./train_20240411/super_epoch_6/run_0/s-6_b-131072_r-0.tf/assets

Epoch 5: ReduceLROnPlateau reducing learning rate to 4.6655999904032795e-05.
INFO:tensorflow:Assets written to: ./train_20240411/super_epoch_6/run_0/s-6_b-131072_r-0.tf/assets
INFO:tensorflow:Assets written to: ./train_20240411/super_epoch_6/run_0/s-6_b-131072_r-0.tf/assets

Epoch 9: ReduceLROnPlateau reducing learning rate to 2.799360081553459e-05.
INFO:tensorflow:Assets written to:

In [19]:
# best_model = './train_20240411/super_epoch_6/run_2/s-6_b-131072_r-2.tf'

K.clear_session() 
print(gc.collect()) # cpu gabage collection

# continue training loop with best model as starting point and smaller batch_sizes, less repeats and lower learning_rate and larger number of (regular) epochs 

model_list, chi2_list, loss_list = DCTR.train_loop(train_data, plt_data, model=best_model, lowest_chi2=lowest_chi2, batch_sizes=[8*8192, 16*8192], repeat=3, super_epochs=4, 
                                                   starting_super_epoch=10, train_dir = train_dir, epochs=25, learning_rate=0.0005)


best_model_list.append(model_list)
lowest_chi2_list.append(chi2_list)

best_model = best_model_list[-1]
lowest_chi2 = lowest_chi2_list[-1]


32112
starting super_epoch 10

starting training with batch_size: 65536 and 25 epochs
starting with weights from model: ./train_20240411/super_epoch_6/run_2/s-6_b-131072_r-2.tf
starting run 0 of super_epoch 10 with batch_size 65536
loaded neural network model: ./train_20240411/super_epoch_6/run_2/s-6_b-131072_r-2.tf
INFO:tensorflow:Assets written to: ./train_20240411/super_epoch_10/run_0/s-10_b-65536_r-0.tf/assets
INFO:tensorflow:Assets written to: ./train_20240411/super_epoch_10/run_0/s-10_b-65536_r-0.tf/assets
INFO:tensorflow:Assets written to: ./train_20240411/super_epoch_10/run_0/s-10_b-65536_r-0.tf/assets
INFO:tensorflow:Assets written to: ./train_20240411/super_epoch_10/run_0/s-10_b-65536_r-0.tf/assets

Epoch 11: ReduceLROnPlateau reducing learning rate to 1.662903669057414e-05.
INFO:tensorflow:Assets written to: ./train_20240411/super_epoch_10/run_0/s-10_b-65536_r-0.tf/assets
INFO:tensorflow:Assets written to: ./train_20240411/super_epoch_10/run_0/s-10_b-65536_r-0.tf/assets
INFO

TypeError: unsupported format string passed to list.__format__

In [ ]:
print(f'best model: {best_model}')
print(f'with mean red chi2 of: {lowest_chi2}')
